In [3]:
# Importing needed libraries
import pandas as pd
import geopandas as gpd

In [161]:
#Load the spatial intersection file generated using spatial outlining notebook
intersection = pd.read_csv('/home/jovyan/work/COVIDRedlining/data/tampa/tampa_redline_intersection.csv')
intersection.head()

,GEOID,polygon_id,weight
0,12057001100,3983.0,0.300331
1,12057001100,3987.0,0.699669
2,12057001200,3983.0,1.000000
3,12057001400,3983.0,1.000000
4,12057001500,3983.0,1.000000


In [162]:
#Load the Redlining Shapefile
redline = gpd.read_file('/home/jovyan/work/COVIDRedlining/data/tampa/tampa redlining/tampa_redlining.shp')
redline.head()

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry
0,3984.0,FL,Tampa,Seminole Heights,A1,A,"{ ""1"" : ""The Section is known as Seminole Heig...","POLYGON ((-82.45523 27.99244, -82.45736 27.992..."
1,3974.0,FL,Tampa,None,A2,A,"{ ""1"" : ""This Section is comprised of several ...","POLYGON ((-82.48380 27.93384, -82.48672 27.930..."
2,3977.0,FL,Tampa,Morrison Grove,A3,A,"{ ""1"" : ""This Section is comprised of the Morr...","MULTIPOLYGON (((-82.48736 27.89578, -82.48606 ..."
3,3989.0,FL,Tampa,Davis Islands,A4,A,"{ ""1"" : ""This is the best residential section ...","POLYGON ((-82.45608 27.92954, -82.45627 27.928..."
4,3983.0,FL,Tampa,Seminole Heights,B1,B,"{ ""1"" : ""This Section, known as Seminole Heigh...","POLYGON ((-82.45038 28.01963, -82.44924 28.019..."


In [163]:
redline['holc_grade'].value_counts()

C    9
D    6
B    5
A    4
Name: holc_grade, dtype: int64

In [164]:
#Convert HOLC Grade to Ordinal Values
def numerical_redlining_value(row):
    if row['holc_grade'] == 'A':
      val = 1
    elif row['holc_grade'] == 'B':
      val = 2
    elif row['holc_grade'] == 'C':
      val = 3
    elif row['holc_grade'] == 'D':
      val = 4
    elif row['holc_grade'] == 'E':
      val = 5
    else:
      val = 0

    return val

redline['numerical_grade'] = redline.apply(numerical_redlining_value, axis=1)

In [165]:
#Merge ACS Shapefile intersection with redline shapefile
#redline['polygon_id'] = redline['polygon_id'].astype(float)
output_combined=pd.merge(redline, intersection, how='left', on='polygon_id')
output_combined.head()

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry,numerical_grade,GEOID,weight
0,3984.0,FL,Tampa,Seminole Heights,A1,A,"{ ""1"" : ""The Section is known as Seminole Heig...","POLYGON ((-82.45523 27.99244, -82.45736 27.992...",1,12057002200,0.229011
1,3984.0,FL,Tampa,Seminole Heights,A1,A,"{ ""1"" : ""The Section is known as Seminole Heig...","POLYGON ((-82.45523 27.99244, -82.45736 27.992...",1,12057001600,0.142376
2,3974.0,FL,Tampa,None,A2,A,"{ ""1"" : ""This Section is comprised of several ...","POLYGON ((-82.48380 27.93384, -82.48672 27.930...",1,12057006000,0.443814
3,3977.0,FL,Tampa,Morrison Grove,A3,A,"{ ""1"" : ""This Section is comprised of the Morr...","MULTIPOLYGON (((-82.48736 27.89578, -82.48606 ...",1,12057005500,0.008171
4,3977.0,FL,Tampa,Morrison Grove,A3,A,"{ ""1"" : ""This Section is comprised of the Morr...","MULTIPOLYGON (((-82.48736 27.89578, -82.48606 ...",1,12057006101,0.604139


In [166]:
#Viewing the shape of the data
output_combined.shape

(104, 11)

In [167]:
output_combined[output_combined['polygon_id'].isna()]

,polygon_id,state,city,name,holc_id,holc_grade,area_descr,geometry,numerical_grade,GEOID,weight


In [168]:
#Removing polygon_ids with NA/NULL values (Check if polygons have corresponding GEOIDs before deleteing)
output_combined = output_combined[output_combined['polygon_id'].notna()]
output_combined.shape

(104, 11)

In [170]:
#Subsetting redline numerical index and spatial weights
redline_index = output_combined[['GEOID','polygon_id','numerical_grade','weight']]
redline_index[redline_index['GEOID']==12057001100]

,GEOID,polygon_id,numerical_grade,weight
9,12057001100,3983.0,2,0.300331
38,12057001100,3987.0,3,0.699669


In [171]:
#Reweighting the redlining numerical index
redline_index['reweighted_redline_index'] = redline_index['numerical_grade'] * redline_index['weight']
redline_index[redline_index['GEOID']==12057001100]

/opt/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,GEOID,polygon_id,numerical_grade,weight,reweighted_redline_index
9,12057001100,3983.0,2,0.300331,0.600662
38,12057001100,3987.0,3,0.699669,2.099006


In [172]:
#Grouping reweighted redline index by GEOID
redline_index_grouped = redline_index[['GEOID','reweighted_redline_index']].groupby('GEOID').sum()
redline_index_grouped[redline_index_grouped.index==12057001100]

,reweighted_redline_index
GEOID,
12057001100,2.699669


In [173]:
#Visualising the dataset
redline_index_grouped.head()

,reweighted_redline_index
GEOID,
12057000700,2.998583
12057000800,3.000000
12057001001,3.000000
12057001100,2.699669
12057001200,2.000000


In [174]:
#Reconverting reweighted values to ordinal
def ordinal_values(row):
    if row['reweighted_redline_index'] > 0 and row['reweighted_redline_index'] <= 1:
      val = 1
    elif row['reweighted_redline_index'] > 1 and row['reweighted_redline_index'] <= 2:
      val = 2
    elif row['reweighted_redline_index'] > 2 and row['reweighted_redline_index'] <= 3:
      val = 3
    elif row['reweighted_redline_index'] > 3 and row['reweighted_redline_index'] <= 4:
      val = 4
    elif row['reweighted_redline_index'] > 4 and row['reweighted_redline_index'] <= 5:
      val = 5
    else:
      val = 0

    return val

redline_index_grouped['ordinal_value'] = redline_index_grouped.apply(ordinal_values, axis=1)

In [175]:
redline_index_grouped.head()

,reweighted_redline_index,ordinal_value
GEOID,,
12057000700,2.998583,3
12057000800,3.000000,3
12057001001,3.000000,3
12057001100,2.699669,3
12057001200,2.000000,2


In [176]:
#Load Census Tract Data
ct_shape = gpd.read_file('/home/jovyan/work/COVIDRedlining/data/tampa/tampa census tracts/tl_2019_12_tract.shp')
ct_shape.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,12,086,013600,12086013600,136,Census Tract 136,G5020,S,782295,29373,+25.8713902,-080.3262779,"POLYGON ((-80.33135 25.87703, -80.33079 25.877..."
1,12,086,013700,12086013700,137,Census Tract 137,G5020,S,1370408,208905,+25.8634748,-080.3283033,"POLYGON ((-80.33949 25.86636, -80.33935 25.866..."
2,12,086,980100,12086980100,9801,Census Tract 9801,G5020,S,14149708,37074872,+25.4728751,-080.1889698,"POLYGON ((-80.26140 25.35472, -80.26139 25.355..."
3,12,075,970200,12075970200,9702,Census Tract 9702,G5020,S,314721147,2351345,+29.4413903,-082.8085416,"POLYGON ((-82.89422 29.53301, -82.89394 29.533..."
4,12,011,110501,12011110501,1105.01,Census Tract 1105.01,G5020,S,987713,15270,+25.9853842,-080.2192385,"POLYGON ((-80.22994 25.98248, -80.22991 25.982..."


In [177]:
#Converting Census Tract Object Datatype to Float
ct_shape['GEOID'] = ct_shape['GEOID'].astype(float)
#Merging redline index scores with Census Tract data
ct_shape = ct_shape.merge(redline_index_grouped,how='inner',on='GEOID')
ct_shape.head()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,reweighted_redline_index,ordinal_value
0,12,057,001100,1.205700e+10,11,Census Tract 11,G5020,S,1631653,45399,+28.0140191,-082.4461143,"POLYGON ((-82.45131 28.01158, -82.45130 28.012...",2.699669,3
1,12,057,001200,1.205700e+10,12,Census Tract 12,G5020,S,1363104,35703,+28.0156612,-082.4586636,"POLYGON ((-82.46488 28.01201, -82.46484 28.012...",2.000000,2
2,12,057,004300,1.205700e+10,43,Census Tract 43,G5020,S,748979,143520,+27.9602984,-082.4711251,"POLYGON ((-82.47671 27.95639, -82.47670 27.957...",4.000000,4
3,12,057,001400,1.205700e+10,14,Census Tract 14,G5020,S,2352954,84840,+28.0037546,-082.4772015,"POLYGON ((-82.48441 27.99684, -82.48439 27.997...",2.000000,2
4,12,057,001500,1.205700e+10,15,Census Tract 15,G5020,S,1465646,85703,+28.0027970,-082.4655029,"POLYGON ((-82.47372 28.00234, -82.47359 28.003...",2.000000,2


In [178]:
#Saving file as geojson
ct_shape.to_file('tampa_redlined_numerical_values.geojson',driver='GeoJSON')